In [1]:
import os
import torch
import numpy as np
import pickle
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tokenizer import Tokenizer
from generation_manager import GeneratorManager

In [2]:
tokenizer = Tokenizer()
print('tokenizer loaded')

curr_model_path = 'ckpt/pretrain_progen_full.pth' # model checkpoint
print('loading model from:', curr_model_path)

tokenizer loaded
loading model from: ckpt/pretrain_progen_full.pth


In [3]:
# paths to the saved .p files
random_selection_file = os.path.join("training_test_data_random_selection_uniParc", "random_selection_100.p")
#data_file = os.path.join("data", "filtered_data_" + query + ".p")

# Reload random_selection from the .p file
random_selection = False
with open(random_selection_file, "rb") as file:
    random_selection = pickle.load(file)

# Reload filtered_data from the .p file
#filtered_data = False
#with open(data_file, "rb") as file:
#    filtered_data = pickle.load(file)

# Print or use the reloaded datasets as needed
print("Random Selection:")
for entry in random_selection:
    print(entry["sequence"]["value"])
    print(len(random_selection))
    print(entry)
    break

#print("---")

#print("Filtered Data:")
#for entry in filtered_data:
#    print(entry["sequence"])'''


Random Selection:
MLLIRLAQVSQEVAATAARSRKTALLAELFREAEADDVPIVIPYLAGRLPQGRLGVGWKVLSRPVPPAAGPTLTVREVDVLLSELGKVSGPGSQAERTRLVGELMGAATEEEQRFLLGLLTGEVRQGALDAVAVEGLARATGAAAADVRRAVMLAGSLQTVAQALLAEGPGALDRFRLTVGRPVLPMLAHSASSVAEAVEKLGACAVEEKLDGIRVQVHRDGDTVRVHTRTLDDITDRLPEVTSAARELRGERFILDGEVISFDAAGRPRSFQETAGRVGSRTDVAKAAGEVPVSPVFFDALSVDGTDLLDLPFAERHAQLARLVPEPMRVRRTVASGPADIAEAERFLADTLARGHEGVVVKALDAPYSAGRRGAAWLKVKPVHTLDLVVLAAEWGHGRRTGKLSNLHLGARSPDGGFAMLGKTFKGMTDAMLAWQTERLQELAVEDNGWVVRVRPELVVEIAYDGLQRSSRYPAGVTLRFARVVRYREDKRPEEADTVQALLAAHPEVTP
100
{'entryType': 'UniProtKB unreviewed (TrEMBL)', 'primaryAccession': 'A0A918IJ85', 'uniProtkbId': 'A0A918IJ85_9ACTN', 'entryAudit': {'firstPublicDate': '2023-02-22', 'lastAnnotationUpdateDate': '2023-06-28', 'lastSequenceUpdateDate': '2023-02-22', 'entryVersion': 3, 'sequenceVersion': 1}, 'annotationScore': 3.0, 'organism': {'scientificName': 'Streptomyces filipinensis', 'taxonId': 66887, 'evidences': [{'evidenceCode': 'ECO:0000313', 'source': 'EMBL', 'id': 'GGV2

In [4]:
import ete3
ncbi = ete3.NCBITaxa()
count = 0
inner_count = 0
for entry in random_selection:
    taxid = int(entry['organism']['taxonId']) # taxonomy id from NCBI
    inner_count = 0
    #print(taxid)
    try:
        tax_lineage1 = ncbi.get_lineage(taxid)
        # remove root taxonomy id
        tax_lineage1 = [element for element in tax_lineage1 if element not in [1, 2, 131567, 10239]]
        #tax_lineage2 = taxa_to_lineage[taxid] # make lineage in ncbi ids
        #print(tax_lineage1)
        res = []
        for ite in tax_lineage1:
            try:
                tmp = taxa_to_ctrl_idx[ite]
                res.append(tmp)
            except:
                inner_count += 1
        if res == [] and inner_count >= 1:
            count += 1
    except:
        count += 1
        print("FATAL taxa conversion")
        #tax_lineage = [taxa_to_ctrl_idx[ite] for ite in tax_lineage1] # now translated as ctrl code indices
print("Tax lineage problems:", count)

Tax lineage problems: 100


<p>To evaluate the generated sequence using teacher forcing:

1. Generate the Sequence: The transformer model with teacher forcing generates the complete sequence. 
At each time step, we provide the true input sequence (+ 1 versus previous step), as input to the model.

2. Calculate Metrics: Accuracy, soft accuracy (based on BLOSUM matrix), perplexity. 

</p>

In [9]:
penalty = 1.2
topk = 3
generator = GeneratorManager(curr_model_path, penalty, topk)

predicted = []
true_tokens_index_in_probs_all = []
tokens_probs_all = []
true_value = []
for entry in random_selection:
    input_seq = entry['sequence']['value']
    taxid = int(entry['organism']['taxonId'])
    try:
        tax_lineage = ncbi.get_lineage(taxid)
        # remove root taxonomy id
        tax_lineage = [element for element in tax_lineage1 if element not in [1, 2, 131567, 10239]]
        res = []
        inner_count = 0
        for ite in tax_lineage:
            try:
                tmp = tokenizer.taxa_to_ctrl_idx[ite]
                res.append(tmp)
            except:
                inner_count += 1
        if res == [] and inner_count >= 1:
            print("ALERT: error in taxonmy conversions for entry: ", taxid)
        tax_lineage = res
    except:
        print("FATAL taxa conversion for entry: ", taxid)
    # Access keywords
    try:
        keywords = [keyword['name'] for keyword in entry['keywords']]
        key_ctrl = []
        for key in keywords:
            ite = tokenizer.name_to_kw[key]
            tmp = tokenizer.kw_to_ctrl_idx[ite]
            key_ctrl.append(tmp)
    except:
        print("FATAL keywords conversion for entry: ", taxid)
    # FOR TESTIG the code:
    # input_seq = input_seq[0:24]
    
    tax_lineage = key_ctrl + tax_lineage

    # if no tax keys:
    # tax_lineage = []
    
    res, tokens_prob, padding = generator.after_n_generation(input_seq, tax_lineage, 20)
        #res, tokens_prob, padding = generator.teacher_forcing_generation(input_seq, tax_lineage)
    
    # print("Input: ", input_seq)
    # print("Res: ", res)
    true_value.append(input_seq)
    true_tokens_index_in_prob = []
    true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
    tokens_probs_all.append(tokens_prob)
    predicted.append(input_seq[:padding] + res)

    # FOR TESTIG:
    break
    
    

MODEL SIZE: 
1280
Found PyTorch checkpoint at  ckpt/pretrain_progen_full.pth
GPU aviable. Previous checkpoint loaded in GPU
FATAL keywords conversion for entry:  66887


In [10]:
print(predicted)
print(true_value)

['MLLIRLAQVSQEVAATAARSLGHQRSGPVASLRGAVLPRGAVPSRGAPVSRGPVSLRAVGSPALRVGSALRVPALSGRPALSRGPAVSRGALPSRGAVPSRGAVSLRGASPLRAGSPVLRGAVPRGLSPAGLSRAPGLRAPSGLRAPSGRLPSGALPRSAVPLSRGALPSGAVLRSAVLPGASRVPALSRGVLAPGVSRALVSRALPVSRGPLSRGVALSRGAVPSRGALPRGSVPRGSALPGVSRAGLPVRGALPVRASLPVRASLPGASRPGASVRGPLSVARPGSLAPGRLSAGRLPASRGVASPGLRASPGRLAPGSLVPAGSLRPVASLRVASGPLRSGALPSVAGRPLVASGPVARGLSPAGVLPARGSVLPARSLGPASLRVPGLAVRPGSVRLGPVSALGPSALRPVASLRGPALRGPSARGPVARGLSVAGRLPSAGVPSRAGLPSARLVPARGSVPLASRGLPSAGRPSVALPSVARGLSPAGVRPSGLRVASPGVASPLVA']
['MLLIRLAQVSQEVAATAARSRKTALLAELFREAEADDVPIVIPYLAGRLPQGRLGVGWKVLSRPVPPAAGPTLTVREVDVLLSELGKVSGPGSQAERTRLVGELMGAATEEEQRFLLGLLTGEVRQGALDAVAVEGLARATGAAAADVRRAVMLAGSLQTVAQALLAEGPGALDRFRLTVGRPVLPMLAHSASSVAEAVEKLGACAVEEKLDGIRVQVHRDGDTVRVHTRTLDDITDRLPEVTSAARELRGERFILDGEVISFDAAGRPRSFQETAGRVGSRTDVAKAAGEVPVSPVFFDALSVDGTDLLDLPFAERHAQLARLVPEPMRVRRTVASGPADIAEAERFLADTLARGHEGVVVKALDAPYSAGRRGAAWLKVKPVHTLDLVVLAAEWGHGRRTGKLSNLHLGARSPDGGFAMLGKTFKGMTDAMLAWQTERLQELAVEDNGWVVRVRPELVVEIAYDGLQRSSRYPAGVTLR

In [ ]:
# Create a directory named "ID_test_data" in the current working directory if it doesn't exist
data_dir = "new_random_selection_100_sequences_p_0_topk_1_kw_taxa_yes_from_50/"
os.makedirs(data_dir, exist_ok=True)

# TODO, salvare tokens_prob e true_tokens_index_in_prob
tokens_probs_data_file = os.path.join(data_dir, "tokens_probs_data.p")
with open(tokens_probs_data_file, "wb") as file:
    pickle.dump(tokens_probs_all, file)
    
true_tokens_index_data_file = os.path.join(data_dir, "true_tokens_index_data.p")
with open(true_tokens_index_data_file, "wb") as file:
    pickle.dump(true_tokens_index_in_probs_all, file)

# Save ID_test_data as a .p file
predicted_data_file = os.path.join(data_dir, "predicted_data.p")
with open(predicted_data_file, "wb") as file:
    pickle.dump(predicted, file)
    
# Save ID_test_data as a .p file
true_data_file = os.path.join(data_dir, "true_data.p")
with open(true_data_file, "wb") as file:
    pickle.dump(true_value, file)
torch.cuda.empty_cache()

In [ ]:
print(input_seq)

In [ ]:
len(input_seq)